In [16]:
import pandas as pd
import numpy as np

In [8]:
# fetch data
data = pd.read_csv('LDPE.csv', usecols = range(1,20)).values
data_train = data[:-4,:] # exclude last 4 faulty samples
data_train

array([[2.08170e+02, 2.96350e+02, 2.33810e+02, 2.83410e+02, 2.39050e+02,
        1.17140e+02, 1.17200e+02, 2.90000e-02, 5.81000e-01, 4.50700e-01,
        4.51800e-01, 6.66420e+02, 2.48950e+02, 3.02100e+03, 1.32200e-01,
        2.73790e+04, 1.60326e+05, 7.81000e-01, 2.61100e+01],
       [2.07260e+02, 2.98260e+02, 2.30880e+02, 2.87550e+02, 2.42550e+02,
        1.16390e+02, 1.17230e+02, 2.80000e-02, 5.74000e-01, 4.76500e-01,
        5.09100e-01, 6.58610e+02, 2.46360e+02, 3.03300e+03, 1.36500e-01,
        2.70430e+04, 1.65044e+05, 8.19000e-01, 2.62900e+01],
       [2.05300e+02, 2.96570e+02, 2.35380e+02, 2.84350e+02, 2.45190e+02,
        1.17330e+02, 1.18420e+02, 3.10000e-02, 5.78000e-01, 4.74400e-01,
        4.50500e-01, 6.66510e+02, 2.44650e+02, 3.00400e+03, 1.33500e-01,
        2.73440e+04, 1.65621e+05, 8.01000e-01, 2.61300e+01],
       [2.09290e+02, 2.94110e+02, 2.25610e+02, 2.83310e+02, 2.42040e+02,
        1.16150e+02, 1.17940e+02, 3.00000e-02, 5.81000e-01, 4.42900e-01,
        4.5160

In [9]:
# scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
data_train_normal = scaler.fit_transform(data_train)

In [10]:
# build PLS model
from sklearn.cross_decomposition import PLSRegression
X_train_normal = data_train_normal[:,:-5]
Y_train_normal = data_train_normal[:,-5:]

In [11]:
pls = PLSRegression(n_components = 3)
pls.fit(X_train_normal, Y_train_normal)

PLSRegression(n_components=3)

In [12]:
# X and Y variance captured
from sklearn.metrics import r2_score

In [13]:
print('Y variance captured: ', 100*pls.score(X_train_normal, Y_train_normal), '%')

Y variance captured:  89.90616771156911 %


In [17]:
Tscores = pls.x_scores_
X_train_normal_reconstruct = np.dot(Tscores, pls.x_loadings_.T)
# can also use pls.inverse_transform(Tscores)

In [18]:
print('X variance captured: ', 100*r2_score(X_train_normal, X_train_normal_reconstruct), '%')

X variance captured:  56.034097079808554 %


Fault detection indices

In [19]:
# monitoring indices for training data T2
T_cov = np.cov(Tscores.T)
T_cov_inv = np.linalg.inv(T_cov)

In [20]:
T2_train = np.zeros((data_train_normal.shape[0],))
for i in range(data_train_normal.shape[0]):
    T2_train[i] = np.dot(np.dot(Tscores[i,:],T_cov_inv),Tscores[i,:].T)

In [21]:
# SPEx
x_error_train = X_train_normal - X_train_normal_reconstruct
SPEx_train = np.sum(x_error_train*x_error_train, axis = 1)

In [22]:
# SPEy
y_error_train = Y_train_normal - pls.predict(X_train_normal)
SPEy_train = np.sum(y_error_train*y_error_train, axis = 1)

In [23]:
# control limits T2
import scipy.stats
N = data_train_normal.shape[0]
k = 3

In [24]:
alpha = 0.01 # 99% control limit
T2_CL = k*(N**2-1)*scipy.stats.f.ppf(1-alpha,k,N-k)/(N*(N-k))

In [25]:
# SPEx
mean_SPEx_train = np.mean(SPEx_train)
var_SPEx_train = np.var(SPEx_train)

In [26]:
g = var_SPEx_train/(2*mean_SPEx_train)
h = 2*mean_SPEx_train**2/var_SPEx_train
SPEx_CL = g*scipy.stats.chi2.ppf(1-alpha, h)

In [27]:
# SPEy
mean_SPEy_train = np.mean(SPEy_train)
var_SPEy_train = np.var(SPEy_train)

In [28]:
g = var_SPEy_train/(2*mean_SPEy_train)
h = 2*mean_SPEy_train**2/var_SPEy_train
SPEy_CL = g*scipy.stats.chi2.ppf(1-alpha, h)

Fault detection on test data

In [29]:
# get test data, normalize it
data_normal = scaler.transform(data)
X_normal = data_normal[:,:-5]
Y_normal = data_normal[:,-5:]

In [30]:
# get model predictions
Tscores_test = pls.transform(X_normal)
X_normal_reconstruct = np.dot(Tscores_test, pls.x_loadings_.T)
Y_normal_pred = pls.predict(X_normal)

In [32]:
#compute monitoring statistics
T2_test = np.zeros((data_normal.shape[0],))
for i in range(data_normal.shape[0]):
    T2_test[i] = np.dot(np.dot(Tscores_test[i,:],T_cov_inv),Tscores_test[i,:].T)

In [33]:
x_error_test = X_normal - X_normal_reconstruct
SPEx_test = np.sum(x_error_test*x_error_test, axis = 1)

In [34]:
y_error_test = Y_normal - pls.predict(X_normal)
SPEy_test = np.sum(y_error_test*y_error_test, axis = 1)